<a href="https://colab.research.google.com/github/2241031c/Night-Lights-Dissertation/blob/main/NL_25_6_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This is an exploratory Notebook using sentinel data to help identify what caused the elevated radiance near Loch Skerrow in April 2020.
This is an area I ended up removing from my analysis due to a forest fire which caused radiance in the area to greatly ecceed the norm.

In [ ]:
# Install necessary packages
!pip install earthengine-api geemap

# Import libraries
import ee
import geemap
import folium

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="######") #enter project number (####)


In [14]:
# Define the Area of Interest (AOI) as a polygon
aoi = ee.Geometry.Polygon(
  [[[-4.14,54.99],
    [-4.16,54.99],
    [-4.20,54.96],
    [-4.18,54.95]]])

# Filter Sentinel-2 Image Collection for April 2020
sentinel_March = ee.ImageCollection('COPERNICUS/S2') \
  .filterDate('2020-02-01', '2020-03-30') \
  .filterBounds(aoi) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Filter Sentinel-2 Image Collection for March 2020
sentinel_April = ee.ImageCollection('COPERNICUS/S2') \
  .filterDate('2020-04-01', '2020-04-30') \
  .filterBounds(aoi) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Check if the collections have images and inspect the bands
print('Sentinel-2 March Collection Size:', sentinel_March.size().getInfo())
print('Sentinel-2 April Collection Size:', sentinel_April.size().getInfo())

# Get the list of band names for an image in each collection to verify
sample_March_image = sentinel_March.first()
sample_August_image = sentinel_April.first()

print('March Image Bands:', sample_March_image.bandNames().getInfo())
print('April Image Bands:', sample_August_image.bandNames().getInfo())

# Calculate median composites if images are present
if sentinel_March.size().getInfo() > 0 and sentinel_April.size().getInfo() > 0:
    median_March = sentinel_March.median().clip(aoi)
    median_April = sentinel_April.median().clip(aoi)

    # Calculate NDVI for both periods
    ndvi_March = median_March.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndvi_April = median_April.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Calculate NDVI difference
    ndvi_diff = ndvi_March.subtract(ndvi_April).rename('NDVI_Diff')

    # Define visualization parameters
    vis_params = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
    rgb_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

    # Create a map centered on the AOI
    Map = geemap.Map(center=[54.96,-4.20], zoom=12)

    # Add layers to the map
    Map.addLayer(median_March, rgb_vis, 'March 2020')
    Map.addLayer(median_April, rgb_vis, 'April 2020')
    Map.addLayer(ndvi_diff, vis_params, 'NDVI Difference')

    # Display the map
    Map
else:
    print("No images found for the specified dates and area.")
Map

Sentinel-2 March Collection Size: 1
Sentinel-2 April Collection Size: 3
March Image Bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']
April Image Bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']


Map(center=[54.96, -4.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [ ]:
# Export the NDVI difference layer as a GeoTIFF to Google Drive
task = ee.batch.Export.image.toDrive(
    image=ndvi_diff,
    description='NDVI_Difference',
    folder='forest_fire_NDVI',  # Replace with your desired folder name
    fileNamePrefix='ndvi_diff',
    region=aoi.bounds(),  # Use the bounds of your AOI
    scale=10,  # Set the desired output resolution in meters
    fileFormat='GeoTIFF'
)

task.start()

#Investigation into Mossdale Galloway 2019   
poi = ee.Geometry.Point(-4.13643, 55.03070).buffer(2000) Mossdale galloway 2019

In [ ]:
# Define the Area of Interest (AOI) as a point
aoi = ee.Geometry.Point(-4.13643, 55.03070).buffer(2000)

# Filter Sentinel-2 Image Collection for April 2020
sentinel_March = ee.ImageCollection('COPERNICUS/S2') \
  .filterDate('2019-02-01', '2019-03-30') \
  .filterBounds(aoi) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Filter Sentinel-2 Image Collection for March 2020
sentinel_August = ee.ImageCollection('COPERNICUS/S2') \
  .filterDate('2019-08-01', '2019-09-30') \
  .filterBounds(aoi) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Check if the collections have images and inspect the bands
print('Sentinel-2 March Collection Size:', sentinel_March.size().getInfo())
print('Sentinel-2 August Collection Size:', sentinel_August.size().getInfo())

# Get the list of band names for an image in each collection to verify
# Check if images exist before trying to get bands
if sentinel_March.size().getInfo() > 0:
    sample_March_image = sentinel_March.first()
    print('March Image Bands:', sample_March_image.bandNames().getInfo())
else:
    print('No images found in Sentinel-2 March Collection')

if sentinel_August.size().getInfo() > 0:
    sample_August_image = sentinel_August.first()
    print('August Image Bands:', sample_August_image.bandNames().getInfo())
else:
    print('No images found in Sentinel-2 August Collection')

# Calculate median composites if images are present
if sentinel_March.size().getInfo() > 0 and sentinel_August.size().getInfo() > 0:
    median_March = sentinel_March.median().clip(aoi)
    median_August = sentinel_August.median().clip(aoi)

    # Calculate NDVI for both periods
    ndvi_March = median_March.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndvi_August = median_August.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Calculate NDVI difference
    ndvi_diff = ndvi_March.subtract(ndvi_August).rename('NDVI_Diff')

    # Define visualization parameters
    vis_params = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
    rgb_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

    # Create a map centered on the AOI
    Map = geemap.Map(center=[55.066, -4.313], zoom=12)

    # Add layers to the map
    Map.addLayer(median_March, rgb_vis, 'March 2020')
    Map.addLayer(median_August, rgb_vis, 'April 2020')
    Map.addLayer(ndvi_diff, vis_params, 'NDVI Difference')

    # Display the map
    Map
else:
    print("No images found for the specified dates and area.")

Sentinel-2 March Collection Size: 4
Sentinel-2 August Collection Size: 3
March Image Bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']
August Image Bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']


In [ ]:
# Filter Sentinel-2 Image Collection for April 2020
sentinel_april = ee.ImageCollection('COPERNICUS/S2') \
  .filterDate('2019-02-01', '2019-03-30') \
  .filterBounds(aoi) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Filter Sentinel-2 Image Collection for March 2020
sentinel_march = ee.ImageCollection('COPERNICUS/S2') \
  .filterDate('2019-08-01', '2019-09-30') \
  .filterBounds(aoi) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Calculate median composites
median_april = sentinel_april.median().clip(aoi)
median_march = sentinel_march.median().clip(aoi)

# Calculate NDVI for both periods
ndvi_april = median_april.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndvi_march = median_march.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Calculate NDVI difference
ndvi_diff = ndvi_april.subtract(ndvi_march).rename('NDVI_Diff')

# Define visualization parameters
vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}
rgb_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

# Create a map centered on the AOI
Map = geemap.Map(center=[54.9, -4.17], zoom=12)

# Add layers to the map
Map.addLayer(median_march, rgb_vis, 'March 2020')
Map.addLayer(median_april, rgb_vis, 'April 2020')
Map.addLayer(ndvi_diff, vis_params, 'NDVI Difference')

# Display the map
Map

Map(center=[54.9, -4.17], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [ ]:
# Export the NDVI difference layer as a GeoTIFF to Google Drive
task = ee.batch.Export.image.toDrive(
    image=ndvi_diff,
    description='NDVI_Difference',
    folder='Mossdale_NDVI',  # Replace with your desired folder name
    fileNamePrefix='ndvi_diff',
    region=aoi.bounds(),  # Use the bounds of your AOI
    scale=10,  # Set the desired output resolution in meters
    fileFormat='GeoTIFF'
)

task.start()